# Codificación de categorías

Las características de nuestros datos a veces se encuentran en forma de etiquetas o categorías. Por ejemplo, la demarcación estatal en donde viven, el nivel educativo o el estado civil. Y recuerda que, a riesgo de sonar repetitivo, los algoritmos de machine learning funcionan con valores numéricos.

In [ ]:
import pandas as pd
import numpy as np

dataset = pd.DataFrame([
 ("Mexico", "Married", "High school"),
 ("Colombia", "Single", "Undergraduate"),
 ("Guinea Equatorial", "Divorced", "College"),
 ("Mexico", "Single", "Primary"),
 ("Colombia", "Single", "Primary"),
], columns=["Country", "Marital status", "Education" ])

dataset

En esta sesión te hablaré de diversas formas en las que podemos codificar valores categóricos para que sean utilizables por algoritmos de machine learning.

## One-hot encoding

Un primer intento de representar las variables categóricas como valores numéricos es usando la codificación <i>One-hot</i> <i>encoding.</i>

En términos simples, el one hot encoding convierte una variable categórica en una matriz de ceros y unos. Cada columna en la matriz representa una un valor único que puede tomar dentro de la categorías de la variable y cada fila representa una observación o muestra. Si una muestra pertenece a una categoría específica, la entrada correspondiente en la matriz será un 1, mientras que todas las demás entradas serán ceros. 

Por ejemplo, tomando nuestro dataset de muestra, vamos a codificar el país utilizando el <i>One-hot encoder</i> de scikit-learn:

Importamos de <code>sklearn.preprocessing</code> y creamos una instancia:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

Y entrenamos nuestro codificador utilizando <code>fit</code> pasándole la columna que queremos codificar:

In [ ]:
encoder.fit(dataset[['Country']])

Y después podemos transformar con <code>transform</code>, por defecto, <code>OneHotEncoder</code> regresa una matriz dispersa, porque en One-hot encoding la matriz resultante está repleta de ceros, así que la convertimos en una matriz densa con <code>todense</code>:

In [ ]:
country_transformed = encoder.transform(dataset[['Country']])
country_transformed.todense()

Puedes ver el orden de las columnas inspeccionando la propiedad <code>categories_</code>:

In [ ]:
encoder.categories_

Y si te das cuenta, estos coinciden con el orden en el que los valores aparecen en la matriz.

### Transformación inversa

Al igual que muchos otros transformadores, <code>OneHotEncoder</code> también tiene el método <code>inverse_transform</code>:

In [ ]:
encoder.inverse_transform(
    np.asarray(country_transformed.todense())
)

### Argumentos extra

La clase <code>OneHotEncoder</code> tiene varios argumentos extra, pero solo considero que hay un par que son importantes para mencionar.

Es comun que entrenes tu codificador con un conjunto de datos, en nuestro caso solamente teníamos tres países en el dataset de entrenamiento, pero ¿qué es lo que va a pasar cuando en el futuro tu modelo reciba otro país? eso es justamente lo que nosotros podemos controlar con el argumento <code>handle_unknown</code>.

Vamos a crear dos codificadores, estableciendo un comportamiento diferente para cada uno. Y de paso vamos a especificar que queremos que nuestro codificador nos entregue por defecto una matriz densa con <code>sparse_output</code>:

In [ ]:
error_encoder = OneHotEncoder(handle_unknown='error', sparse_output=False)
ignore_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

Después los entrenamos con nuestros datos existentes

In [ ]:
error_encoder.fit(dataset[['Country']])
ignore_encoder.fit(dataset[['Country']])

Y veamos qué es lo que sucede cuando intentamos probarlos con datos nuevos:

In [ ]:
new_data = pd.DataFrame(['Costa Rica'], columns=['Country'])

Primero hay que intentar el del error. Y de hecho lo voy a poner en un bloque <i>try-except</i> para agarrar el error – es importante destacar que este es el comportamiento por defecto.

In [ ]:
try:
	error_encoder.transform(new_data)
except ValueError as ve:
	print(ve)

Si intentamos con el que le hemos dicho que lo ignore, nos regresará puros ceros puesto que lo ignora:

In [ ]:
ignore_encoder.transform(new_data)

### ¿Cuándo utilizar <code>OneHotEncoder</code>?

Es bueno utilizar esta herramienta cuando nuestras categorías no tienen un orden predefinido, como el caso de los países, no podemos definir cuál es mayor que el otro, ni por más patrióticos que nos pongamos.

## Ordinal encoding

Hay otro tipo de variables que si nos permiten codificar cierta noción de orden y jerarquía, como es el caso de las variables categóricas ordinales. Piensa en el grado de estudio dentro de nuestro dataset.

Dependiendo del problema que estemos enfrentando, podemos definir que el haber cursado la primaria es menos que haber cursado la educación superior.

Para reflejar este tipo de relaciones podemos utilizar el <code>OrdinalEncoder</code>:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

Y creamos un objeto de la clase, pasándole como argumento las categorías que puede tomar nuestra variable en el orden que queramos que sean tomadas en cuenta – si no se establecen, los números serán asigandos al azar:

In [ ]:
ordinal_encoder = OrdinalEncoder(categories=[[
 "Primary", "Secondary", "High school", "Undergraduate", "College"
]])

Y ahora entonces podemos entrenar el codificador:

In [ ]:
ordinal_encoder.fit(dataset[['Education']])

Y al transformar el dataset obtenemos lo esperado:

In [ ]:
ordinal_encoder.transform(dataset[['Education']])

### Argumentos extra

Al igual que el codificador <i>one-hot</i>, <code>OrdinalEncoder</code> tiene varios argumentos extra, pero quizá el más importante es el que especifica cómo comportarse ante información no vista antes.

Vamos a experimentar con los dos valores posibles, <code>error</code> y <code>use_encoded_value</code>:

In [ ]:
error_encoder = OrdinalEncoder(categories=[[
 "Primary", "Secondary", "High school", "Undergraduate", "College"
]], handle_unknown='error')

error_encoder.fit(dataset[['Education']])

De nuevo, para manejar el error hay que ponerlo en un bloque <i>try-except</i>:

In [ ]:
try:
	error_encoder.transform([["Kindergarten"]])
except ValueError as ve:
	print(ve)

Por otro lado, si creamos uno que utilize el valor por defecto, podemos utilizar <code>handle_unknown</code> a <code>use_encoded_value</code>, para el caso, también es necesario establecer el argumento <code>unknown_value</code>:

In [ ]:
default_encoder = OrdinalEncoder(categories=[[
 "Primary", "Secondary", "High school", "Undergraduate", "College"
]],
 handle_unknown='use_encoded_value',
unknown_value=np.nan)

default_encoder.fit(dataset[['Education']])

Y si intentamos transformar un valor que no existía previamente:

In [ ]:
default_encoder.transform([["Kindergarten"]])

En donde recibirá el valor de <code>np.nan</code> por defecto en lugar de fallar.

### ¿Cuándo es mejor utilizar <code>OrdinalEncoder</code>?

Utiliza ordinal encoder cuando tus variables tengan un sentido de orden entre ellas, así podrás preservarlo para cuando conviertas de cadenas a números.

 > 📚 Tanto <code>OrdinalEncoder</code> como <code>OneHotEncoder</code> permiten ser entrenados en más de una columna a la vez, ¿qué te parece si codificas el estado civil de los datos al mismo tiempo que cualquiera de los otros dos? mejor aún, ¿qué codificador hace más sentido usar para ese atributo de nuestros datos?